## Stock Analysis ##

This is a project to create an interactive report using Plotly and Datapane

## Data Souce ## 
Yahoo Finace

In [61]:
import pandas as pd
import numpy as np
import plotly.figure_factory as ff
import datapane as dp
import plotly.offline as py
import plotly.graph_objs as go
import plotly.express as px
import pandas_datareader.data as web
import datetime

# py.init_notebook_mode(connected=True)

import plotly.io as pio
pio.templates.default = "plotly_dark"

start = datetime.datetime(2020, 8, 1)
end = datetime.datetime.now()

btc = web.DataReader("BTC-USD", 'yahoo', start, end).reset_index()
eth = web.DataReader("ETH-USD", 'yahoo', start, end).reset_index()

We'll be creating all of our plots using Plotly

In [36]:
trace0 = go.Scatter(x = btc.Date, y = btc.Close, name = 'BTC-USD')
fig0 = go.Figure([trace0])
fig0.update_layout(
    title = {
        'text': "Bitcoin-USD Stock Price",
        'x':0.5,
        'xanchor': 'center'})
fig0.show()

In [37]:
btc['10-day MA'] = btc['Close'].rolling(window=10).mean()
btc['20-day MA'] = btc['Close'].rolling(window=20).mean()
btc['50-day MA'] = btc['Close'].rolling(window=50).mean()

trace0 = go.Scatter(x = btc.Date, y = btc.Close, name = 'BTC-USD')
trace1 = go.Scatter(x = btc.Date, y = btc['10-day MA'], name = '10-day MA')
trace2 = go.Scatter(x = btc.Date, y = btc['20-day MA'], name = '20-day MA')

fig1 = go.Figure([trace0, trace1, trace2])

fig1.update_layout(
    title={
        'text': "Bitcoin-USD Stock Price",
        'x':0.5,
        'xanchor': 'center'})

fig1.show()

In [38]:
fig2 = go.Figure(go.Candlestick(x=btc.Date, open=btc.Open, high= btc.High, low=btc.Low, close=btc.Close))

fig2.update_layout(
    title={
        'text' : "Bitcoint-USD Stock Price (Candle Stick)",
        'x' : 0.5,
        'xanchor':'center'})

fig2.show()

In [39]:
btc['Daily return (%)'] = round(btc['Close'].pct_change()*100,2)

fig3 = px.bar(btc, x = "Date", y = "Daily return (%)")
fig3.update_layout(title={
    'text': "Bitcoint-USD Stock Daily Return",
    'x' : 0.5,
    'xanchor':'center'})

fig3.show()

In [40]:
trace0 = go.Scatter(x=btc.Date, y=btc.Close, name='BTC-USD', line=dict(color='lime'))
trace1 = go.Scatter(x=eth.Date, y=eth.Close, name='ETH-USD', line=dict(color='red'))

fig4 = go.Figure([trace0, trace1])

fig4.update_layout(dict(
    title=dict({
        'text': "Bitcoin-USD vs Ethereum-USD Stock Price",
        'x':0.5,
        'xanchor': 'center'}),
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label='1m',
                     step='month',
                     stepmode='backward'),
                dict(count=3,
                     label='3m',
                     step='month',
                     stepmode='backward'),
                dict(count=6,
                     label='6m',
                     step='month',
                     stepmode='backward'),
                dict(count=12,
                     label='12m',
                     step='month',
                     stepmode='backward')
            ])
        ),
        rangeslider=dict(
            visible = True
        ),
        type='date')))

fig4.show()

In [45]:
trace0 = go.Scatterpolar(
    r = [btc['Close'].mean(), btc['Open'].min(), btc['Low'].min(), btc['High'].max()], 
    theta = ['Close', 'Open', 'Low', 'High'], line = dict(color ='lime'), 
    name='BTC-USD', fill = 'toself'
    )

trace1 = go.Scatterpolar(
      r = [eth['Close'].mean(),eth['Open'].min(), eth['Low'].min(),eth['High'].max()],
      theta = ['Close','Open','Low','High'], line =  dict(
            color = 'red'), name='ZM',
      fill = 'toself'
    )

fig5 = go.Figure([trace0, trace1])

fig5.update_layout(go.Layout(
      polar = dict(
        radialaxis = dict(
          visible = True)),
            title=dict({
        'text': "Bitcoin-USD vs Ethereum-USD Stock Price (Radar Chart)",
        'x':0.5,
        'xanchor': 'center'})))
fig5.show()

In [57]:
df = web.DataReader(['XRP-USD', 'ETH-USD', 'BNB-USD', 'DOGE-USD'], 'yahoo', start, end)['Close']

trace0 = go.Box(y=df['XRP-USD'], name='XRP-USD', line=dict(color='blue'))
trace1 = go.Box(y=df['ETH-USD'], name='ETH-USD', line=dict(color='red'))
trace2 = go.Box(y=df['BNB-USD'], name='BNB-USD', line=dict(color='green'))
trace3 = go.Box(y=df['DOGE-USD'], name='DOGE-USD', line=dict(color='purple'))

fig6 = go.Figure([trace0, trace1, trace2, trace3])

fig6.update_layout(
    title={
        'text': "Criptocurrency Prices",
        'x':0.5,
        'xanchor': 'center'})

fig6.show()

In [64]:
fig7 = px.area(df, facet_col="Symbols", facet_col_wrap=2)
fig7.update_layout(
    title={
        'text': "Stock Prices",
        'x':0.5,
        'xanchor': 'center'})
fig7.show()

In [62]:
btc = btc[['Open', 'Close', 'Volume']]

btc["index"] = np.arange(len(btc))

fig8 = go.Figure(ff.create_scatterplotmatrix(btc, diag='box', index='index',size=3,
                               height=600, width=1150, colormap='RdBu',
                                   title={
                                    'text': "Bitcoin-USD Stock Price (Scatterplot Matrix)",
                                    'x':0.5,
                                    'xanchor': 'center'}))
fig8.show()

## Publishing our report ##

Now that we have a series of plots created using Plotly, we can create a report using Datapane. Although we could create a linear list of all plots (similar to how they are displayed in this notebook), we are choosing to use Datapane's Blocks component to create a grid layout, similar to what you might see in a BI tool.


In [66]:
dp.Report(
    dp.Group(
        dp.Plot(fig0), 
        dp.Plot(fig1), 
        dp.Plot(fig2), 
        dp.Plot(fig3), 
        dp.Plot(fig4), 
        dp.Plot(fig5), 
        dp.Plot(fig6), 
        dp.Plot(fig7), 
        columns=2,
        rows=4
    ), dp.Plot(fig8)
).publish(name='stock_report', open=True)

Publishing document and associated data - *please wait...*

Your report doesn't contain any text - did you know you can add text to your report once published?

Report successfully published at https://datapane.com/u/rafael32sf/reports/stock-report/ - you can edit and add additional text from the link